In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd
from tqdm import tqdm
from transformers import BertTokenizerFast, BertForSequenceClassification, get_linear_schedule_with_warmup, AdamW
from sklearn.model_selection import train_test_split
import torch
import os
import numpy as np
import random
from sklearn.metrics import accuracy_score
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, models
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
model = torch.load('../model_bert_best.pth') #Сюда нужно вставить ссылку на модель BERT с гугл - диска.
raw_model = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(raw_model, do_lower_case=True)

In [ ]:

## Step 1: use an existing language model
word_embedding_model = models.Transformer('distilroberta-base')
#word_embedding_model = models.Transformer('stsb-roberta-large')
## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

In [7]:
model_cos = torch.load("../model_cos_5_64") #Сюда нужно вставить ссылку на модель SBERT с гугл - диска.

In [8]:
def convert_to_dataset(data: pd.DataFrame) -> TensorDataset:
    input_ids = []
    attention_masks = []
    token_type_ids = []
    for _, row in tqdm(data.iterrows(), total=data.shape[0]):
        encoded_dict = tokenizer.encode_plus(row["query"], row["name"], max_length=300,
                                             pad_to_max_length=True,
                                             return_attention_mask=True, return_tensors='pt', truncation=True)
        input_ids.append(encoded_dict['input_ids'])
        token_type_ids.append(encoded_dict["token_type_ids"])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    input_ids.to(dtype=torch.long)
    token_type_ids.to(dtype=torch.long)
    attention_masks.to(dtype=torch.long)

    return TensorDataset(input_ids, attention_masks, token_type_ids)

In [9]:
def infer(dataloader, model):
    embs = []

    for batch in tqdm(dataloader, desc="Evaluating", unit="batch"):
        input_ids, attention_masks, token_type_ids = batch

        input_ids = input_ids.to(dtype=torch.long)
        token_type_ids = token_type_ids.to(dtype=torch.long)
        attention_masks = attention_masks.to(dtype=torch.long)
        with torch.no_grad():
            m = (model(input_ids,
                       token_type_ids=token_type_ids,
                       attention_mask=attention_masks)).logits
            embs.append(torch.nn.Softmax()(m))
    return embs

In [10]:
df = pd.read_csv("../train.csv") #Сюда нужно вставить ссылку на данные с гугл - диска.

In [ ]:
all_comp = {}
for i, row in df.iterrows():
  all_comp[row["name_1"]] = {"emb": None}
  all_comp[row["name_2"]] = {"emb": None}

print(len(all_comp))
json.dump(all_comp, open("../all_comp.json", "w"), indent=4)

In [ ]:
all_comp = json.load(open("../all_comp.json")) #Сюда нужно вставить ссылку на данные с гугл - диска.

for sent, d in tqdm(all_comp.items(), total=len(all_comp)):
  emm = model_cos.encode([sent])
  all_comp[sent] = {"emb": emm[0]}

print(len(all_comp))

In [36]:
def mesure_cos(target_company, all_comp_emb, n=10):
  target_emb = model_cos.encode(target_company)[0]
  output = pd.DataFrame(columns=['company_name', 'sim_score'])

  comp_labels = []
  comp_emb = []
  for comp, emd_dict in all_comp_emb.items():
      comp_labels.append(comp)
      comp_emb.append(emd_dict["emb"])
  
  out = cosine_similarity([target_emb], comp_emb)
  for i, score in enumerate(out[0]):
    output.loc[len(output)] = [comp_labels[i], score]
  
  output = output.sort_values(by=['sim_score'], ascending=False)

  print(output.head(10))

  best = output.iloc[:n+1]
  top_n = []
  for i, b in best.iterrows():
    top_n.append(b["company_name"])

  return top_n

In [17]:
target_company = ["API"]

In [37]:
output = mesure_cos(target_company, all_comp)


                                           company_name  sim_score
5651                                                API   1.000000
4996                                        Trinseo API   0.811521
4865                                             A.P.I.   0.793274
2343      A.P.I. Applicazioni Plastiche Industriali SPA   0.689926
6341                                    ASIA PHARM PACK   0.670014
16168                                   SIA"INTER SPED"   0.665998
9906                           Active Print & Promotion   0.663481
765                                                 APS   0.659733
6574   A.P.I. APPLICAZIONI PLASTICHE INDUSTRIALI S.P.A.   0.656669
6380                                Performance Machine   0.641626


In [38]:
output

['API',
 'Trinseo API',
 'A.P.I.',
 'A.P.I. Applicazioni Plastiche Industriali SPA',
 'ASIA PHARM PACK',
 'SIA"INTER SPED"',
 'Active Print & Promotion',
 'APS',
 'A.P.I. APPLICAZIONI PLASTICHE INDUSTRIALI S.P.A.',
 'Performance Machine',
 'Active Screw & Fastener']

In [41]:
target_company = [target_company[0] for i in range(len(output))]

In [42]:
df = pd.DataFrame({'query': target_company,
                   'name': output})

In [44]:
test = convert_to_dataset(df)
test_dataloader = DataLoader(test, sampler=SequentialSampler(test), batch_size=1)
emn = infer(test_dataloader, model)
ems = [i.detach().cpu().numpy() for i in emn]
em = [(ems[i][0][1]) for i, k in enumerate(ems)]
df['is_duplicate_score'] = pd.Series(em)
df = df.sort_values('is_duplicate_score', ascending=False)
print(df)

  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Evaluating:   0%|          | 0/11 [00:00<?, ?batch/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
Evaluating: 100%|██████████| 11/11 [00:13<00:00,  1.22s/batch]

   query                                              name  is_duplicate_score
1    API                                       Trinseo API            0.999756
0    API                                               API            0.999727
3    API     A.P.I. Applicazioni Plastiche Industriali SPA            0.995598
8    API  A.P.I. APPLICAZIONI PLASTICHE INDUSTRIALI S.P.A.            0.994212
2    API                                            A.P.I.            0.992988
5    API                                   SIA"INTER SPED"            0.038426
10   API                           Active Screw & Fastener            0.000190
7    API                                               APS            0.000081
9    API                               Performance Machine            0.000077
6    API                          Active Print & Promotion            0.000045
4    API                                   ASIA PHARM PACK            0.000026
